In [ ]:
import json

In [ ]:
import cloudscraper
import geopandas as gpd
import pandas as pd

In [3]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [6]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [7]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [8]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Raley's Data

In [12]:
scraper = cloudscraper.create_scraper()

In [43]:
url = "https://www.raleys.com/api/store"
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7",
    "content-type": "application/json",
    "priority": "u=1, i",
    "request-id": "|600220e7299d4632942091e3d5cc84e1.76fb5acc74ef47b5",
    "sec-ch-ua": '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "traceparent": "00-600220e7299d4632942091e3d5cc84e1-76fb5acc74ef47b5-01",
    "x-csrf-token": "5f0a22add84aa57769510b7710061a35dbf6f10531590751dc0094a46fc0401c",
    "x-referrer-path": "",
    "x-url-path": "/stores",
    "cookie": "",  # Get cookie from https://www.raleys.com/stores
    "Referer": "https://www.raleys.com/stores",
    "Referrer-Policy": "strict-origin-when-cross-origin",
}
body = {
    "offset": 0,
    "rows": 200,
    "searchParameter": {
        "shippingMethod": "pickup",
        "searchString": "",
        "latitude": "",
        "longitude": "",
        "maxMiles": 99999,
        "departmentIds": [],
    },
}

In [44]:
r = scraper.post(url, headers=headers, data=json.dumps(body))

In [48]:
stores_json = r.json()["data"]

In [56]:
store_dicts = []
for store in stores_json:
    state_code = store["address"]["state"]
    brand = store["brand"]["name"]
    point = Point(store["coordinates"]["longitude"], store["coordinates"]["latitude"])
    store_dict = {"geometry": point, "brand": brand, "STUSPS": state_code}
    store_dicts.append(store_dict)

In [57]:
raleys_gdf = gpd.GeoDataFrame(store_dicts, crs=4326)

In [58]:
raleys_gdf.to_file("data/raleys.gpkg")

In [62]:
raleys_counts_df = (
    pd.DataFrame(raleys_gdf.groupby("STUSPS").size())
    .reset_index()
    .rename(columns={0: "stores"})
)

## Combine With States

In [65]:
raleys_counts_gdf = states_with_population_df.merge(
    raleys_counts_df, on="STUSPS", how="left"
)

In [66]:
raleys_counts_gdf = raleys_counts_gdf.fillna(0)
raleys_counts_gdf["stores"] = raleys_counts_gdf["stores"].astype(int)

In [67]:
raleys_counts_gdf["per_100k"] = raleys_counts_gdf["stores"] / (
    raleys_counts_gdf["POPULATION"] / 100000
)
raleys_counts_gdf["per_1m"] = raleys_counts_gdf["stores"] / (
    raleys_counts_gdf["POPULATION"] / 1_000_000
)

In [68]:
raleys_counts_gdf = raleys_counts_gdf.fillna(0)

In [69]:
raleys_counts_gdf = raleys_counts_gdf.to_crs(9311)
raleys_counts_gdf.to_file(f"data/raleys_per_states.gpkg")